In [2]:
import pandas
as pd

In [3]:
df = pd.read_csv("Spamdata.csv")

In [4]:
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [5]:
import sklearn
print(sklearn.__version__)

1.6.1


In [6]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ahars\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [8]:
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(df["text"])
y = df["label_num"]

In [9]:
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size=0.2,random_state=42)

In [10]:
model1 = MultinomialNB()
model1.fit(x_train,y_train)

MultinomialNB()

In [11]:
model2 = LogisticRegression(C=1.0,solver="lbfgs")
model2.fit(x_train,y_train)

LogisticRegression()

In [12]:
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.01,0.1,1],
    'kernel': ['rbf']
}

In [13]:
svc = SVC()
model3 = GridSearchCV(svc, param_grid, cv=5, verbose=2, n_jobs=-1)
model3.fit(x_train,y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1],
                         'kernel': ['rbf']},
             verbose=2)

In [14]:
# print("Best parameters: ",model3.best_params_)
# print("Best cross-val score",model3.best_score_)
# best_model = model3.best_estimator_

# y_pred = best_model.predict(X_test)

# print("🎯 Test Accuracy:", accuracy_score(y_test, y_pred))
# print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))
# print("\n🧱 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [37]:
# def spam_checking(new_email):
#     x_new = vectorizer.transform([new_email])
#     return x_new
# def naive_bayes(new_email):
#     x_new = spam_checking(new_email)
#     prediction1 = model1.predict(x_new)
#     prob1 = model1.predict_proba(x_new)[0][1]
#     if(prediction1[0] == 1):
#         print("Naive bayes: Spam")
#     else:
#         print("Naive bayes: Not Spam")
    
#     print("Spam confidence using naive bayes: ",prob1)
  
# def logistic_regression(new_email):
#     x_new = spam_checking(new_email)
#     prediction2 = model2.predict_proba(x_new)[0][1]
#     if(prediction2 >= 0.56):
#         print("logistic regression: Spam")
#     else:
#         print("logistic regression: Not Spam")

#     print("Spam confidence using logistic regression: ",prediction2)
# def svm(new_email):
#     x_new = spam_checking(new_email)
#     prediction3 = model3.predict(x_new)
#     decision_score = model3.decision_function(x_new)[0]
#     import numpy as np
#     confidence = 1 / (1 + np.exp(-decision_score))  # Sigmoid to scale it
    

#     if(prediction3[0] == 1):
#         print("SVM: Spam")
#     else:
#         print("SVM: Not Spam")
#     print("SVM confidence:", confidence)
# new_email = input("enter the email: ")
# naive_bayes(new_email)
# logistic_regression(new_email)
# svm(new_email)

enter the email:  hurray you won  the lottery!


Naive bayes: Not Spam
Spam confidence using naive bayes:  0.4626671745018731
logistic regression: Spam
Spam confidence using logistic regression:  0.634786783344377
SVM: Spam
SVM confidence: 0.6335680039669922


In [51]:
def spam_checking(new_email):
    x_new = vectorizer.transform([new_email])
    return x_new
def naive_bayes(new_email):
    x_new = spam_checking(new_email)
    prediction1 = model1.predict(x_new)
    confidence1 = model1.predict_proba(x_new)[0][1]
    return prediction1[0],confidence1
  
def logistic_regression(new_email):
    x_new = spam_checking(new_email)
    confidence2 = model2.predict_proba(x_new)[0][1]
    if(confidence2 >= 0.56):
        prediction2 = 1
    else:
        prediction2 = 0

    return prediction2,confidence2
def svm(new_email):
    x_new = spam_checking(new_email)
    prediction3 = model3.predict(x_new)
    decision_score = model3.decision_function(x_new)[0]
    import numpy as np
    confidence3 = 1 / (1 + np.exp(-decision_score))  # Sigmoid to scale it
    return prediction3[0], confidence3;

new_email = input("enter the email: ")
nb_pred, nb_conf = naive_bayes(new_email)
lr_pred, lr_conf = logistic_regression(new_email)
svm_pred, svm_conf = svm(new_email)
spam_votes = nb_pred+lr_pred+svm_pred
avg_conf = (nb_conf + lr_conf + svm_conf) / 3
if new_email.strip() != "":
    if(spam_votes >= 2):
        print("SPAM SPAM SPAM")
        print("Spam confidence:",round(avg_conf*100,2),"%")
    else:
        print("NOT SPAM")
        print("NOT Spam confidence:",round((100 - (avg_conf*100)),2),"%")

enter the email:  hello


SPAM SPAM SPAM
Spam confidence: 73.88 %


In [16]:
import joblib
joblib.dump(model1,'naive_bayes_model.joblib')
joblib.dump(model2,'logistic_regression_model.joblib')
joblib.dump(vectorizer,'vectorizer.joblib')
joblib.dump(model3,'svm.joblib')

['svm.joblib']